### Общий план:
- __Заполнить пропуски в данных диагностики__
- __Агрегировать результаты диагностики по каждому юзеру__
- Выделить фичи из текста
- Агрегировать фичи по юзеру
- Объединить датасеты
- Построить матрицу корреляций по полученному датасету
- Наиболее удачные, яркие корреляции рассмотреть подробнее
- Построить бейзлайн, лр/бустинг, для прогнозирования результата диагностики по фичам текста

In [123]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from functools import reduce
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

Работаем с дополнительными данными рефлексии, предоставленными организаторами.

In [11]:
# для воспроизведения - изменить путь!

path = r"\Дополнительные данные\Reflection_data+"

### Возьмем только анкеты с ответами на вопрос 'Чему Вы научились в рамках курса?'

In [61]:
add_data = []
for file in os.listdir(os.path.join(path)):
    df = pd.read_excel(os.path.join(path,file))
#     print(df.columns[1])
    if df.columns[1] == 'Чему Вы научились в рамках курса?' or df.columns[1] == 'Чему вы научились в рамках курса?':
        df.columns = ['untiID', 'Чему вы научились в рамках курса?']
        add_data.append(df)

In [62]:
# таких анкет - 51 из 69
len(add_data)

51

In [63]:
add_data = pd.concat(add_data)

In [64]:
add_data.shape

(1510, 2)

In [65]:
add_data.isna().sum()

untiID                                0
Чему вы научились в рамках курса?    29
dtype: int64

In [66]:
add_data

,untiID,Чему вы научились в рамках курса?
0,701335,"Оформила страницу в социальной сети, определил..."
1,1033016,До начала курса я немного был знаком с основам...
2,995121,Очень много новой информации. Научились состав...
3,674661,"Очень полезный курс, научилась грамотно вести ..."
4,969910,научилась очень многому благодаря компетенции ...
...,...,...
26,1034455,Курсы по этой программе весьма интересны и име...
27,290243,Узнал много инструментов для разработки прилож...
28,1002173,"Да очень, много познавательного интересного уз..."
29,984791,"Есть полезная информация, которую стоит держат..."


### Объединим с данными диагностик

In [85]:
# для воспроизведения - изменить путь!
path_diag = r"\Дополнительные данные"

data_diag = pd.read_excel(os.path.join(path_diag, r'Диагностика_ПЦС_доп..xlsx'),header=1)
data_diag.shape

(20613, 68)

In [86]:
data_diag.rename(columns={"Leader ID": "untiID"}, inplace=True)

In [87]:
data_diag.columns

Index(['untiID', 'Прогресс прохождения обязательных активностей',
       'Количество пройденных обязательных активностей',
       'Количество пройденных дополнительных активностей', 'автоматизация',
       'единая идеология компании', 'поощрение инаковости',
       'исследование неопределённости', 'поощрение работы над ошибками',
       'нестандартные решения', 'гибкий график, удалённая работа',
       'мало гибких правил', 'открытая система управления знаниями',
       'кооперация', 'откровенность', 'демократия',
       'краткосрочное планирование', 'эффективность', 'инновации',
       'вовлечённость', 'динамичные команды', 'обучение желающих',
       'власть экспертизы', 'коллективизм', 'актер', 'адепт', 'аналитик',
       'боец', 'визионер', 'вождь', 'интуит', 'конструктор', 'контролер',
       'эксперт', 'организатор', 'исполнитель', 'преследователь',
       'экспериментатор', 'связной', 'творец', 'хакер', 'хозяин', 'эмпат',
       'эстет', 'осмысленность деятельности, собственная 

In [93]:
data = pd.merge(add_data ,data_diag, on=['untiID'])

### Заполнение пропусков в данных диагностики

In [106]:
data.dtypes

untiID                                                                                                                               object
Чему вы научились в рамках курса?                                                                                                    object
Прогресс прохождения обязательных активностей                                                                                       float64
Количество пройденных обязательных активностей                                                                                      float64
Количество пройденных дополнительных активностей                                                                                    float64
                                                                                                                                     ...   
Количество кликов на любой символ на желтом фоне                                                                                    float64
Количество кликов на

In [117]:
data.fillna(0, inplace=True)

In [118]:
sum(data.isna().sum())

0

In [124]:
def aggregate(data):
    sum_df = data.groupby(['untiID'], as_index=False).agg(np.sum)
    sum_df.columns = [str(col) + '_sum' if col != 'untiID' else 'untiID' for col in sum_df.columns]
    
    mean_df = data.groupby(['untiID'], as_index=False).agg(np.mean)
    mean_df.columns = [str(col) + '_mean' if col != 'untiID' else 'untiID' for col in mean_df.columns]
    
    median_df = data.groupby(['untiID'], as_index=False).agg(np.median)
    median_df.columns = [str(col) + '_median' if col != 'untiID' else 'untiID' for col in median_df.columns]
    
    max_df = data.groupby(['untiID'], as_index=False).agg(np.max)
    max_df.columns = [str(col) + '_max' if col != 'untiID' else 'untiID' for col in max_df.columns]
    
    min_df = data.groupby(['untiID'], as_index=False).agg(np.min)
    min_df.columns = [str(col) + '_min' if col != 'untiID' else 'untiID' for col in min_df.columns]
    
    data_frames = [sum_df, mean_df, median_df, max_df, min_df]
#     df_merged = pd.merge(sum_df, mean_df, on=['Unti ID'])
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['untiID'], how='right'), data_frames)
    return df_merged

In [125]:
data_agg = aggregate(data.drop('Чему вы научились в рамках курса?',axis=1))

In [126]:
data_agg.shape

(1245, 327)

In [137]:
# Сохраним агрегированные диагностические доп.данные
data_agg.to_csv('additional_data_agg.csv',index=False)

In [159]:
# Сохраним агрегированные текстовые доп.данные
data[['untiID', 'Чему вы научились в рамках курса?']].to_csv('additional_data_text.csv',index=False)

### Объединим основной трейн (диагностики db0-7) и доп.данные

In [140]:
diag_train_agg = pd.read_csv('diag_train_agg.csv')

In [141]:
diag_train_agg.shape

(206, 681)

In [149]:
# check if there are the same ids
print("Кол-во совпадающих ID:" , data_agg.shape[0] - len(set(data_agg.untiID) - set(diag_train_agg.untiID)))

Кол-во совпадающих ID: 0


In [164]:
# Определим совпадающие колонки
# Все колонки доп.данных есть в основном трейне - объединим датасеты по ним.
set(data_agg.columns) - set(diag_train_agg.columns)

set()

In [172]:
diag_large_train = pd.concat([data_agg, diag_train_agg[data_agg.columns.tolist()]])
diag_large_train.shape

(1451, 327)

In [171]:
# sanity check
diag_large_train['творец_mean'].value_counts()

 0.0    1123
 1.0     240
 2.0      62
-1.0      19
 3.0       5
-2.0       2
Name: творец_mean, dtype: int64

In [174]:
# сохраним трейн датасет с результатами диагностик
diag_large_train.to_csv('diag_large_train.csv',index=False)